# Creating Crosswalk with DLA Locode, NTD ID and ITP ID

In [1]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [4]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/'

pd.set_option('display.max_columns', None)

In [5]:
dla = pd.read_excel(f"{GCS_FILE_PATH}locodes_updated7122021.xlsx")

In [6]:
dla = to_snakecase(dla)
dla = dla.rename(columns = {'agency_name':'name', 'agency_locode':'ct_code'})

In [7]:
dla.sample(1)

ct_code      name  district    county_name  \
837     5236  Plymouth        10  Amador County   

                                   rtpa_name mpo_name mpo_locode_fads  \
837  Amador County Transportation Commission  NON-MPO         NON-MPO   

    active_e76s______7_12_2021_  
837                         Yes

In [8]:
dla = dla>>select(_.ct_code, _.name, _.district)

### Reading in [Place Names](https://dot.ca.gov/-/media/dot-media/programs/research-innovation-system-information/documents/place-names/2020-place-names-in-california-a11y.pdf)

In [9]:
pl_city = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=0)
pl_county = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=1)

In [10]:
pl_city = to_snakecase(pl_city)
pl_county = to_snakecase(pl_county)

In [11]:
pl_city = pl_city.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_7', 'date_of_incorporation', 'urban_area_no_'})
pl_county = pl_county.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_9'})

In [12]:
pl_city = pl_city.rename(columns={'ct_city_code':'ct_code', 'city_name_abbr_':'abbr', 'dist_':'district', 'co_':'county'})
pl_county = pl_county.rename(columns={'ct_co__code':'ct_code', 'county_name':'name', 'co__name_abbr_':'abbr'})

In [13]:
pl_city['organization_type']= ('City/Town')
pl_county['organization_type']= ('County')

In [14]:
pl_county['name'] = pl_county['name'].astype(str) + ' County' 

In [15]:
pl_county.ct_code.fillna(pl_county['unnamed:_8'], inplace=True)
pl_county = pl_county.drop(columns = {'unnamed:_8'})

In [16]:
plnames = pd.concat([pl_city, pl_county])

In [17]:
plnames

name  abbr district county  ct_code organization_type  co__fips
0          Adelanto  Adel        8    SBd   5408.0         City/Town       NaN
1      Agoura Hills  AgrH        7     LA   5435.0         City/Town       NaN
2           Alameda   Ala        4    Ala   5014.0         City/Town       NaN
3            Albany   Alb        4    Ala   5178.0         City/Town       NaN
4          Alhambra   Alh        7     LA   5130.0         City/Town       NaN
..              ...   ...      ...    ...      ...               ...       ...
53    Tulare County   Tul        6    NaN   5946.0            County     107.0
54  Tuolumne County   Tuo       10    NaN   5932.0            County     109.0
55   Ventura County   Ven        7    NaN   5952.0            County     111.0
56      Yolo County   Yol        3    NaN   5922.0            County     113.0
57      Yuba County   Yub        3    NaN   5916.0            County     115.0

[540 rows x 7 columns]

In [18]:
plnames.county.fillna(plnames.abbr, inplace=True)

In [19]:
fips_map = dict(zip(pl_county['abbr'], 
                          pl_county['co__fips']))

In [20]:
plnames['co__fips'] = plnames['county'].map(fips_map)

In [21]:
type_map = dict(zip(plnames['ct_code'], 
                          plnames['organization_type']))

In [22]:
dla['organization_type'] = dla['ct_code'].map(type_map)

In [23]:
mask1 = (dla['name'].str.contains('Council of Governments'))
mask2 = (dla['name'].str.contains('Tribe')) 
mask3 = (dla['name'].str.contains('University'))
mask4 = (dla['name'].str.contains('RTPA'))
mask5 = (dla['name'].str.contains('JPA'))

dla.loc[dla['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
dla.loc[dla['organization_type'].isnull() & mask2, 'organization_type'] = 'Tribe'
dla.loc[dla['organization_type'].isnull() & mask3, 'organization_type'] = 'University - Public'
dla.loc[dla['organization_type'].isnull() & mask4, 'organization_type'] = 'MPO/RTPA'
dla.loc[dla['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'

In [1]:
#dla.to_csv('dla_placenames.csv')

### READ ITP

In [24]:
itp = pd.read_csv(f"{GCS_FILE_PATH}organizations_All_Organizations.csv")

In [25]:
itp = to_snakecase(itp)

In [26]:
itp = itp>>select(_.name, _.ntp_id, _.itp_id, _.organization_type)

In [27]:
itp = itp.rename(columns = {"ntp_id":"ntd_id"})

In [28]:
itp

name      ntd_id  itp_id  \
0                                        A-Paratransit         NaN     NaN   
1                                          ABC Shuttle         NaN     NaN   
2                                            Able Inc.         NaN     NaN   
3                                         Abrazar Inc.         NaN     NaN   
4                                      Access Services       90157     1.0   
..                                                 ...         ...     ...   
823       Yosemite Area Regional Transportation System  9R02-91070   374.0   
824                      Yuba-Sutter Transit Authority       90061   376.0   
825  Yuma County Intergovernmental Public Transport...       90233   386.0   
826                                        Yurok Tribe       99262   377.0   
827                                         Zonar Inc.         NaN     NaN   

           organization_type  
0                        NaN  
1                    Company  
2    Non-Profit Organization  
3    Non-Profit Organization  
4                        NaN  
..                       ...  
823       Independent Agency  
824       Independent Agency  
825       Independent Agency  
826                    Tribe  
827                  Company  

[828 rows x 4 columns]

## Merging

In [29]:
ids = pd.merge(itp, dla, on='name', how="outer", indicator=True)

In [30]:
ids._merge.value_counts()

right_only    924
left_only     711
both          117
Name: _merge, dtype: int64

* right only is DLA
* left only is ITP/NTD ID

In [31]:
ids>>count(_.name)>>arrange(-_.n)>>filter(_.n>1)

name   n
187                   Caltrans  13
1096          North East Trees   2
1650  Urban Corps of San Diego   2

In [32]:
ids>>filter(_.name=='North East Trees')

name ntd_id  itp_id organization_type_x  ct_code  district  \
1367  North East Trees    NaN     NaN                 NaN   6118.0       7.0   
1368  North East Trees    NaN     NaN                 NaN   6434.0       7.0   

     organization_type_y      _merge  
1367                 NaN  right_only  
1368                 NaN  right_only

In [33]:
ids>>filter(_.name=='Urban Corps of San Diego')

name ntd_id  itp_id organization_type_x  ct_code  \
1617  Urban Corps of San Diego    NaN     NaN                 NaN   6490.0   
1618  Urban Corps of San Diego    NaN     NaN                 NaN   6233.0   

      district organization_type_y      _merge  
1617      11.0                 NaN  right_only  
1618      11.0                 NaN  right_only

In [34]:
#duplicates expected
ids>>filter(_.name=='Caltrans')

name ntd_id  itp_id organization_type_x  ct_code  district  \
845  Caltrans    NaN     NaN                 NaN   6201.0       1.0   
846  Caltrans    NaN     NaN                 NaN   6202.0       2.0   
847  Caltrans    NaN     NaN                 NaN   6203.0       3.0   
848  Caltrans    NaN     NaN                 NaN   6204.0       4.0   
849  Caltrans    NaN     NaN                 NaN   6205.0       5.0   
850  Caltrans    NaN     NaN                 NaN   6206.0       6.0   
851  Caltrans    NaN     NaN                 NaN   6207.0       7.0   
852  Caltrans    NaN     NaN                 NaN   6208.0       8.0   
853  Caltrans    NaN     NaN                 NaN   6209.0       9.0   
854  Caltrans    NaN     NaN                 NaN   6210.0      10.0   
855  Caltrans    NaN     NaN                 NaN   6211.0      11.0   
856  Caltrans    NaN     NaN                 NaN   6212.0      12.0   
857  Caltrans    NaN     NaN                 NaN   6200.0      53.0   

    organization_type_y      _merge  
845                 NaN  right_only  
846                 NaN  right_only  
847                 NaN  right_only  
848                 NaN  right_only  
849                 NaN  right_only  
850                 NaN  right_only  
851                 NaN  right_only  
852                 NaN  right_only  
853                 NaN  right_only  
854                 NaN  right_only  
855                 NaN  right_only  
856                 NaN  right_only  
857                 NaN  right_only

In [35]:
#interesting that DLA Locode has some duplciates... will investigate
#caltrans duplicates expected

#### Adding Organization Types

In [36]:
ids.organization_type_y.fillna(ids['organization_type_x'], inplace=True)

In [37]:
ids = ids.rename(columns={'organization_type_y':'organization_type'})
ids.drop(columns=('organization_type_x'), inplace=True)

In [38]:
ids

name ntd_id  itp_id  ct_code  district  \
0                          A-Paratransit    NaN     NaN      NaN       NaN   
1                            ABC Shuttle    NaN     NaN      NaN       NaN   
2                              Able Inc.    NaN     NaN      NaN       NaN   
3                           Abrazar Inc.    NaN     NaN      NaN       NaN   
4                        Access Services  90157     1.0   6312.0       7.0   
...                                  ...    ...     ...      ...       ...   
1747                         Leave Blank    NaN     NaN   5465.0      53.0   
1748      U.S. Fish and Wildlife Service    NaN     NaN   6250.0      53.0   
1749  Tidewater Southern Railway Company    NaN     NaN   6031.0      53.0   
1750    Tri-Counties Regional Park Group    NaN     NaN   6176.0      53.0   
1751               Trust for Public Land    NaN     NaN   6185.0      59.0   

            organization_type      _merge  
0                         NaN   left_only  
1                     Company   left_only  
2     Non-Profit Organization   left_only  
3     Non-Profit Organization   left_only  
4                         NaN        both  
...                       ...         ...  
1747                      NaN  right_only  
1748                      NaN  right_only  
1749                      NaN  right_only  
1750                      NaN  right_only  
1751                      NaN  right_only  

[1752 rows x 7 columns]

In [39]:
mask1 = (ids['name'].str.contains('Council of Governments'))
mask2 = (ids['name'].str.contains('Association of Governments')) 
mask4 = (ids['name'].str.contains('Transportation Commission'))
mask5 = (ids['name'].str.contains('Joint Powers Agency'))
mask6 = (ids['name'].str.contains('Transportation Agency'))
mask8 = (ids['name'].str.contains('Port Hueneme'))
mask9 = (ids['name'].str.contains('Access Services'))
mask10 = (ids['name'].str.contains('County Transportation Authority'))


ids.loc[ids['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
ids.loc[ids['organization_type'].isnull() & mask2, 'organization_type'] = 'Council of Governments'
ids.loc[ids['organization_type'].isnull() & mask4, 'organization_type'] = 'Transportation Commission'
ids.loc[ids['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'
ids.loc[ids['organization_type'].isnull() & mask6, 'organization_type'] = 'MPO/RTPA'
ids.loc[ids['organization_type'].isnull() & mask8, 'organization_type'] = 'City/Town'
ids.loc[ids['organization_type'].isnull() & mask9, 'organization_type'] = 'Independent Agency'
ids.loc[ids['organization_type'].isnull() & mask10, 'organization_type'] = 'Transportation Commission'

In [40]:
mask11 = (ids['name'].str.contains('Regional'))

In [41]:
ids.organization_type.value_counts()

City/Town                    755
Company                      150
Non-Profit Organization       93
Independent Agency            75
County                        58
Transportation Commission     39
University - Public           34
Council of Governments        22
Tribe                          7
Joint Powers Agency            6
MPO/RTPA                       6
University - Private           5
Federal Government             2
Name: organization_type, dtype: int64

In [42]:
(
    ids
    >> filter(_.organization_type.isnull())
    >> filter(_.name.str.contains("Governments"))
)

name ntd_id  itp_id  ct_code  \
382       Council of San Benito County Governments    NaN     NaN   6060.0   
1227  Association Of Monterey Bay Area Governments    NaN     NaN   6091.0   
1260          Council of Fresno County Governments    NaN     NaN   6086.0   
1463      Western Riverside Council Of Governments    NaN     NaN   6107.0   
1464         San Bernardino Associated Governments    NaN     NaN   6053.0   

      district organization_type      _merge  
382        5.0               NaN        both  
1227       5.0               NaN  right_only  
1260       6.0               NaN  right_only  
1463       8.0               NaN  right_only  
1464       8.0               NaN  right_only

#### Subset for Orgs that matched

In [43]:
both = (ids>>filter(_._merge=='both'))

In [44]:
both = both.drop(columns=['_merge'])

In [45]:
both

name      ntd_id  itp_id  ct_code  \
4                             Access Services       90157     1.0   6312.0   
13                              Alpine County  9R02-91116     9.0   5931.0   
15                              Amador County         NaN     NaN   5926.0   
16    Amador County Transportation Commission         NaN     NaN   6127.0   
28          Antelope Valley Transit Authority       90121    16.0   6166.0   
..                                        ...         ...     ...      ...   
798  Ventura County Transportation Commission       90164   380.0   6155.0   
803           Victor Valley Transit Authority       90148   360.0   6261.0   
815    Western Contra Costa Transit Authority       90159   368.0   6226.0   
822       Yolo County Transportation District       90090   372.0   6195.0   
824             Yuba-Sutter Transit Authority       90061   376.0   6224.0   

     district          organization_type  
4         7.0         Independent Agency  
13       10.0                     County  
15       10.0                     County  
16       10.0  Transportation Commission  
28        7.0         Independent Agency  
..        ...                        ...  
798       7.0  Transportation Commission  
803       8.0         Independent Agency  
815       4.0         Independent Agency  
822       3.0         Independent Agency  
824       3.0         Independent Agency  

[117 rows x 6 columns]

In [46]:
both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 4 to 824
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               117 non-null    object 
 1   ntd_id             56 non-null     object 
 2   itp_id             66 non-null     float64
 3   ct_code            117 non-null    float64
 4   district           117 non-null    float64
 5   organization_type  111 non-null    object 
dtypes: float64(3), object(3)
memory usage: 6.4+ KB


In [47]:
both['organization_type'].value_counts()

Independent Agency           42
County                       36
Transportation Commission    15
Council of Governments       12
Joint Powers Agency           3
MPO/RTPA                      2
City/Town                     1
Name: organization_type, dtype: int64

In [48]:
nomatch = ids>>filter(_._merge!='both')

In [49]:
nomatch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1635 entries, 0 to 1751
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   name               1635 non-null   object  
 1   ntd_id             143 non-null    object  
 2   itp_id             308 non-null    float64 
 3   ct_code            924 non-null    float64 
 4   district           924 non-null    float64 
 5   organization_type  1141 non-null   object  
 6   _merge             1635 non-null   category
dtypes: category(1), float64(3), object(3)
memory usage: 91.1+ KB


In [50]:
nomatch>>count(_._merge)

_merge    n
0   left_only  711
1  right_only  924
2        both    0

In [51]:
(nomatch.query('itp_id.notnull() and ntd_id.notnull()')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 10 to 826
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   name               135 non-null    object  
 1   ntd_id             135 non-null    object  
 2   itp_id             135 non-null    float64 
 3   ct_code            0 non-null      float64 
 4   district           0 non-null      float64 
 5   organization_type  128 non-null    object  
 6   _merge             135 non-null    category
dtypes: category(1), float64(3), object(3)
memory usage: 7.6+ KB


In [52]:
nomatch.query('itp_id.isnull() or ntd_id.isnull() or ct_code.isnull()')

name ntd_id  itp_id  ct_code  district  \
0                          A-Paratransit    NaN     NaN      NaN       NaN   
1                            ABC Shuttle    NaN     NaN      NaN       NaN   
2                              Able Inc.    NaN     NaN      NaN       NaN   
3                           Abrazar Inc.    NaN     NaN      NaN       NaN   
5                            ACE Parking    NaN     NaN      NaN       NaN   
...                                  ...    ...     ...      ...       ...   
1747                         Leave Blank    NaN     NaN   5465.0      53.0   
1748      U.S. Fish and Wildlife Service    NaN     NaN   6250.0      53.0   
1749  Tidewater Southern Railway Company    NaN     NaN   6031.0      53.0   
1750    Tri-Counties Regional Park Group    NaN     NaN   6176.0      53.0   
1751               Trust for Public Land    NaN     NaN   6185.0      59.0   

            organization_type      _merge  
0                         NaN   left_only  
1                     Company   left_only  
2     Non-Profit Organization   left_only  
3     Non-Profit Organization   left_only  
5                         NaN   left_only  
...                       ...         ...  
1747                      NaN  right_only  
1748                      NaN  right_only  
1749                      NaN  right_only  
1750                      NaN  right_only  
1751                      NaN  right_only  

[1635 rows x 7 columns]

In [53]:
no_ids = nomatch>>filter(_.itp_id.isnull())>>filter(_.ntd_id.isnull())

In [54]:
no_ids>>filter(_.ct_code.isnull())

name ntd_id  itp_id  \
0                                        A-Paratransit    NaN     NaN   
1                                          ABC Shuttle    NaN     NaN   
2                                            Able Inc.    NaN     NaN   
3                                         Abrazar Inc.    NaN     NaN   
5                                          ACE Parking    NaN     NaN   
..                                                 ...    ...     ...   
817  Westminster on Wheels Senior Transportation Pr...    NaN     NaN   
818                               Willits Seniors Inc.    NaN     NaN   
820                                           Worldpay    NaN     NaN   
821                                      Yellow Cab CA    NaN     NaN   
827                                         Zonar Inc.    NaN     NaN   

     ct_code  district        organization_type     _merge  
0        NaN       NaN                      NaN  left_only  
1        NaN       NaN                  Company  left_only  
2        NaN       NaN  Non-Profit Organization  left_only  
3        NaN       NaN  Non-Profit Organization  left_only  
5        NaN       NaN                      NaN  left_only  
..       ...       ...                      ...        ...  
817      NaN       NaN                      NaN  left_only  
818      NaN       NaN  Non-Profit Organization  left_only  
820      NaN       NaN                  Company  left_only  
821      NaN       NaN                  Company  left_only  
827      NaN       NaN                  Company  left_only  

[395 rows x 7 columns]

In [55]:
both.query('itp_id.isnull() or ntd_id.isnull()')

name ntd_id  itp_id  ct_code  \
15                               Amador County    NaN     NaN   5926.0   
16     Amador County Transportation Commission    NaN     NaN   6127.0   
55                                Butte County    NaN     NaN   5912.0   
59            Calaveras Council of Governments    NaN     NaN   6128.0   
371    Colusa County Transportation Commission    NaN     NaN   6129.0   
..                                         ...    ...     ...      ...   
759  Transportation Agency for Monterey County    NaN     NaN   6143.0   
765   Trinity County Transportation Commission    NaN     NaN   6153.0   
768                              Tulare County    NaN     NaN   5946.0   
770   Tulare County Association of Governments    NaN     NaN   6094.0   
774     Tuolumne County Transportation Council    NaN     NaN   6154.0   

     district          organization_type  
15       10.0                     County  
16       10.0  Transportation Commission  
55        3.0                     County  
59       10.0     Council of Governments  
371       3.0  Transportation Commission  
..        ...                        ...  
759       5.0                   MPO/RTPA  
765       2.0  Transportation Commission  
768       6.0                     County  
770       6.0        Joint Powers Agency  
774      10.0                        NaN  

[63 rows x 6 columns]

In [56]:
both.query('organization_type.isnull()')

name ntd_id  itp_id  \
382           Council of San Benito County Governments    NaN     NaN   
498             Lake County/City Area Planning Council    NaN     NaN   
608       Placer County Transportation Planning Agency    NaN     NaN   
694  Shasta County Regional Transportation Planning...    NaN     NaN   
739                     Tahoe Regional Planning Agency    NaN     NaN   
774             Tuolumne County Transportation Council    NaN     NaN   

     ct_code  district organization_type  
382   6060.0       5.0               NaN  
498   6136.0       1.0               NaN  
608   6158.0       3.0               NaN  
694   6093.0       2.0               NaN  
739   6125.0       3.0               NaN  
774   6154.0      10.0               NaN

In [57]:
# Most Agency types that are null are Transportation Commissions, or COGs. 

In [58]:
print(f"Out of {(len(ids>>filter(_.ct_code.notnull())))} organizations in the Official Locode List, {(len(ids>>filter(_.ct_code.notnull())>>filter(_.organization_type.notnull())))} have an Organization Type listed in Airtable")


Out of 1041 organizations in the Official Locode List, 659 have an Organization Type listed in Airtable


In [59]:
ids.organization_type.value_counts()

City/Town                    755
Company                      150
Non-Profit Organization       93
Independent Agency            75
County                        58
Transportation Commission     39
University - Public           34
Council of Governments        22
Tribe                          7
Joint Powers Agency            6
MPO/RTPA                       6
University - Private           5
Federal Government             2
Name: organization_type, dtype: int64

In [60]:
# There is a `Council of Governments` option available 

In [61]:
(both.query('organization_type.isnull()'))

name ntd_id  itp_id  \
382           Council of San Benito County Governments    NaN     NaN   
498             Lake County/City Area Planning Council    NaN     NaN   
608       Placer County Transportation Planning Agency    NaN     NaN   
694  Shasta County Regional Transportation Planning...    NaN     NaN   
739                     Tahoe Regional Planning Agency    NaN     NaN   
774             Tuolumne County Transportation Council    NaN     NaN   

     ct_code  district organization_type  
382   6060.0       5.0               NaN  
498   6136.0       1.0               NaN  
608   6158.0       3.0               NaN  
694   6093.0       2.0               NaN  
739   6125.0       3.0               NaN  
774   6154.0      10.0               NaN

In [62]:
both>>filter(_.ntd_id.isnull())

name ntd_id  itp_id  ct_code  \
15                               Amador County    NaN     NaN   5926.0   
16     Amador County Transportation Commission    NaN     NaN   6127.0   
55                                Butte County    NaN     NaN   5912.0   
59            Calaveras Council of Governments    NaN     NaN   6128.0   
371    Colusa County Transportation Commission    NaN     NaN   6129.0   
..                                         ...    ...     ...      ...   
759  Transportation Agency for Monterey County    NaN     NaN   6143.0   
765   Trinity County Transportation Commission    NaN     NaN   6153.0   
768                              Tulare County    NaN     NaN   5946.0   
770   Tulare County Association of Governments    NaN     NaN   6094.0   
774     Tuolumne County Transportation Council    NaN     NaN   6154.0   

     district          organization_type  
15       10.0                     County  
16       10.0  Transportation Commission  
55        3.0                     County  
59       10.0     Council of Governments  
371       3.0  Transportation Commission  
..        ...                        ...  
759       5.0                   MPO/RTPA  
765       2.0  Transportation Commission  
768       6.0                     County  
770       6.0        Joint Powers Agency  
774      10.0                        NaN  

[61 rows x 6 columns]

In [63]:
both>>filter(_.itp_id.isnull())

name      ntd_id  itp_id  \
15                                       Amador County         NaN     NaN   
16             Amador County Transportation Commission         NaN     NaN   
55                                        Butte County         NaN     NaN   
59                    Calaveras Council of Governments         NaN     NaN   
371            Colusa County Transportation Commission         NaN     NaN   
382           Council of San Benito County Governments         NaN     NaN   
408         El Dorado County Transportation Commission         NaN     NaN   
433                      Fresno Council of Governments         NaN     NaN   
434                                      Fresno County         NaN     NaN   
446             Glenn County Transportation Commission         NaN     NaN   
465         Humboldt County Association of Governments         NaN     NaN   
470                                    Imperial County         NaN     NaN   
480                                        Inyo County         NaN     NaN   
496            Kings County Association of Governments         NaN     NaN   
498             Lake County/City Area Planning Council         NaN     NaN   
500            Lassen County Transportation Commission         NaN     NaN   
519            Madera County Transportation Commission         NaN     NaN   
532                   Mendocino Council of Governments         NaN     NaN   
535           Merced County Association of Governments         NaN     NaN   
547                                        Mono County         NaN     NaN   
549                                    Monterey County         NaN     NaN   
566            Nevada County Transportation Commission         NaN     NaN   
588                                      Orange County         NaN     NaN   
608       Placer County Transportation Planning Agency         NaN     NaN   
612                                      Plumas County  9R02-91119     NaN   
613            Plumas County Transportation Commission         NaN     NaN   
617                                       Port Hueneme         NaN     NaN   
632                                   Riverside County         NaN     NaN   
647                                  San Benito County         NaN     NaN   
649                              San Bernardino County         NaN     NaN   
652                                   San Diego County         NaN     NaN   
663                 San Joaquin Council of Governments         NaN     NaN   
664                                 San Joaquin County         NaN     NaN   
670                             San Luis Obispo County         NaN     NaN   
672                                   San Mateo County         NaN     NaN   
674                               Santa Barbara County         NaN     NaN   
678                                  Santa Cruz County         NaN     NaN   
679  Santa Cruz County Regional Transportation Comm...         NaN     NaN   
693                                      Shasta County  9R02-99438     NaN   
694  Shasta County Regional Transportation Planning...         NaN     NaN   
709                    Solano Transportation Authority         NaN     NaN   
717     Southern California Association of Governments         NaN     NaN   
728                  Stanislaus Council of Governments         NaN     NaN   
729                                  Stanislaus County         NaN     NaN   
739                     Tahoe Regional Planning Agency         NaN     NaN   
742            Tehama County Transportation Commission         NaN     NaN   
759          Transportation Agency for Monterey County         NaN     NaN   
765           Trinity County Transportation Commission         NaN     NaN   
768                                      Tulare County         NaN     NaN   
770           Tulare County Association of Governments         NaN     NaN   
774             Tuolumne County Transportation Council         NaN     N

In [64]:
# many in both have blank NTD and ITP ids but are still identified in Airtable

In [65]:
print(f"Out of the {(len(both))} organizations that matched, {(len(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()')))} have all id fields filled (ITP/NTD/Locode)")


Out of the 117 organizations that matched, 54 have all id fields filled (ITP/NTD/Locode)


In [66]:
#the 54 with all fields filled out
(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()'))

name      ntd_id  itp_id  \
4                                      Access Services       90157     1.0   
13                                       Alpine County  9R02-91116     9.0   
28                   Antelope Valley Transit Authority       90121    16.0   
56             Butte County Association of Governments       90208    47.0   
60                                    Calaveras County  9R02-91063    50.0   
407                 El Dorado County Transit Authority       90229   101.0   
435                 Fresno County Rural Transit Agency  9R02-91007   117.0   
445                                       Glenn County  9R02-91088   122.0   
451                     Golden Empire Transit District       90004   126.0   
468                         Humboldt Transit Authority  9R02-91036   135.0   
471          Imperial County Transportation Commission       90226   138.0   
493                                        Kern County  9R02-91059   146.0   
495            Kings County Area Public Transit Agency       90200   148.0   
499                             Lake Transit Authority  9R02-91053   159.0   
502                      Lassen Transit Service Agency  9R02-91098   162.0   
510  Los Angeles County Metropolitan Transportation...       90154   182.0   
518                                      Madera County  9R02-91005   188.0   
522                      Marin County Transit District       90234   194.0   
533                        Mendocino Transit Authority  9R02-91047   198.0   
554                    Morongo Basin Transit Authority  9R02-91090   212.0   
555           Mountain Area Regional Transit Authority  9R02-91012   214.0   
562               Napa Valley Transportation Authority       90088   218.0   
565                                      Nevada County  9R02-91095   221.0   
572                      North County Transit District       90030   226.0   
580                                          OmniTrans       90029   232.0   
589             Orange County Transportation Authority       90036   235.0   
607                                      Placer County       90196   251.0   
616             Pomona Valley Transportation Authority       99425    69.0   
626                    Redwood Coast Transit Authority  9R02-91097   261.0   
633         Riverside County Transportation Commission       90218   267.0   
634                           Riverside Transit Agency       90031   269.0   
642               Sacramento Regional Transit District       90019   273.0   
651               San Diego Association of Governments       90095   276.0   
654              San Diego Metropolitan Transit System       90026   278.0   
658      San Francisco Bay Area Rapid Transit District       90003   279.0   
666               San Joaquin Regional Rail Commission       90182    10.0   
669             San Luis Obispo Council of Governments       90297   288.0   
673                  San Mateo County Transit District       90009   290.0   
676        Santa Barbara Metropolitan Transit District       90020   293.0   
677        Santa Clara Valley Transportation Authority       90013   294.0   
680           Santa Cruz Metropolitan Transit District       90006   296.0   
699            Sierra County Transportation Commission  9R02-91038   304.0   
704                                    Siskiyou County  9R02-91048    83.0   
708                              Solano County Transit       90232   310.0   
711                                      Sonoma County       90089   314.0   
718        Southern California Regional Rail Authority       90151   323.0   
741                                      Tehama County  9R02-91089   334.0   
764                                     Trinity County  9R02-91035   344.0   
797                                     Ventura County       90298    84.0   
798           Ventura County Transportation Commission       90164   380.0   
803                    Victor Valley Transit Authority       90148   360

In [67]:
print(f"If we just look at completed Locodes (no ITP/NTD), we now have organization type info for {(len(both.query('ct_code.notnull() and organization_type.notnull()')))} organizations")


If we just look at completed Locodes (no ITP/NTD), we now have organization type info for 111 organizations


In [68]:
#both.to_parquet(f"{GCS_FILE_PATH}ITP_NTD_Locode_crosswalk.parquet")

#this just has the Agency Locode, NTD ID, ITP ID and Agency Locode

## Full Merged Data

In [69]:
ids

name ntd_id  itp_id  ct_code  district  \
0                          A-Paratransit    NaN     NaN      NaN       NaN   
1                            ABC Shuttle    NaN     NaN      NaN       NaN   
2                              Able Inc.    NaN     NaN      NaN       NaN   
3                           Abrazar Inc.    NaN     NaN      NaN       NaN   
4                        Access Services  90157     1.0   6312.0       7.0   
...                                  ...    ...     ...      ...       ...   
1747                         Leave Blank    NaN     NaN   5465.0      53.0   
1748      U.S. Fish and Wildlife Service    NaN     NaN   6250.0      53.0   
1749  Tidewater Southern Railway Company    NaN     NaN   6031.0      53.0   
1750    Tri-Counties Regional Park Group    NaN     NaN   6176.0      53.0   
1751               Trust for Public Land    NaN     NaN   6185.0      59.0   

            organization_type      _merge  
0                         NaN   left_only  
1                     Company   left_only  
2     Non-Profit Organization   left_only  
3     Non-Profit Organization   left_only  
4          Independent Agency        both  
...                       ...         ...  
1747                      NaN  right_only  
1748                      NaN  right_only  
1749                      NaN  right_only  
1750                      NaN  right_only  
1751                      NaN  right_only  

[1752 rows x 7 columns]

In [70]:
print(f"Althought we still have {len(ids>>filter(_.organization_type.isnull()))} without Organization Types")

Althought we still have 500 without Organization Types


In [71]:
ids>>filter(_.name.str.contains('Council'))>>select(_.name, _.organization_type)

name        organization_type
59             Calaveras Council of Governments   Council of Governments
71                    Cambria Community Council  Non-Profit Organization
382    Council of San Benito County Governments                      NaN
433               Fresno Council of Governments   Council of Governments
492                 Kern Council of Governments   Council of Governments
498      Lake County/City Area Planning Council                      NaN
532            Mendocino Council of Governments   Council of Governments
639      Sacramento Area Council of Governments   Council of Governments
662                         San Joaquin Council                      NaN
663          San Joaquin Council of Governments   Council of Governments
669      San Luis Obispo Council of Governments   Council of Governments
684   Saratoga Area Senior Coordinating Council  Non-Profit Organization
728           Stanislaus Council of Governments   Council of Governments
774      Tuolumne County Transportation Council                      NaN
802    Victor Valley Community Services Council  Non-Profit Organization
897           Mohawk Valley Stewardship Council                      NaN
1022     Oakland Private Industry Council, Inc.                      NaN
1260       Council of Fresno County Governments                      NaN
1304         Kern County Council of Governments   Council of Governments
1316  San Gabriel Valley Council of Governments   Council of Governments
1350    South Bay Cities Council of Governments   Council of Governments
1446      Gateway Cities Council of Governments   Council of Governments
1463   Western Riverside Council Of Governments                      NaN
1661       Orange County Council of Governments   Council of Governments
1705       Council for Tribal Employment Rights                      NaN

In [72]:
#all
ids

name ntd_id  itp_id  ct_code  district  \
0                          A-Paratransit    NaN     NaN      NaN       NaN   
1                            ABC Shuttle    NaN     NaN      NaN       NaN   
2                              Able Inc.    NaN     NaN      NaN       NaN   
3                           Abrazar Inc.    NaN     NaN      NaN       NaN   
4                        Access Services  90157     1.0   6312.0       7.0   
...                                  ...    ...     ...      ...       ...   
1747                         Leave Blank    NaN     NaN   5465.0      53.0   
1748      U.S. Fish and Wildlife Service    NaN     NaN   6250.0      53.0   
1749  Tidewater Southern Railway Company    NaN     NaN   6031.0      53.0   
1750    Tri-Counties Regional Park Group    NaN     NaN   6176.0      53.0   
1751               Trust for Public Land    NaN     NaN   6185.0      59.0   

            organization_type      _merge  
0                         NaN   left_only  
1                     Company   left_only  
2     Non-Profit Organization   left_only  
3     Non-Profit Organization   left_only  
4          Independent Agency        both  
...                       ...         ...  
1747                      NaN  right_only  
1748                      NaN  right_only  
1749                      NaN  right_only  
1750                      NaN  right_only  
1751                      NaN  right_only  

[1752 rows x 7 columns]